# L'Analyse Factorielle des Correspondances (AFC)

#### run pour installer

In [79]:
#%pip install pandas
#%pip install fanalysis
#%pip install numpy
#%pip install matplotlib

In [39]:
pip install fanalysis

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [40]:
import pandas as pd
from fanalysis.ca import CA
import numpy as np
import matplotlib.pyplot as plt

In [4]:
pip install xlrd

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
data=pd.read_excel("C:\\Users\\khadi\\Downloads\\train (1).xlsx")
data

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,LP002978,Female,No,0,Graduate,No,2900,0.0,71,360,1,Rural,Y
496,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40,180,1,Rural,Y
497,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253,360,1,Urban,Y
498,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187,360,1,Urban,Y


In [7]:
# Vérification des valeurs manquantes
missing_values = data.isnull().sum()

# Affichage des valeurs manquantes par colonne
print(missing_values)



Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [16]:
df = df.set_index("Loan_Status")
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
Loan_Status,,,,,,,,,,,,
N,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128,360,1,Rural
Y,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66,360,1,Urban
Y,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120,360,1,Urban
Y,LP001008,Male,No,0,Graduate,No,6000,0.0,141,360,1,Urban
Y,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267,360,1,Urban
...,...,...,...,...,...,...,...,...,...,...,...,...
Y,LP002978,Female,No,0,Graduate,No,2900,0.0,71,360,1,Rural
Y,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40,180,1,Rural
Y,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253,360,1,Urban


In [19]:
#Afficher les types de données
print("Types de données avant conversion :\n", df.dtypes)

Types de données avant conversion :
 Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount             int64
Loan_Amount_Term       int64
Credit_History         int64
Property_Area         object
dtype: object


In [20]:
# Convertir toutes les colonnes en numérique
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [24]:
# Afficher les types de données après conversion
print("Types de données après conversion :\n", df.dtypes)

Types de données après conversion :
 Loan_ID              float64
Gender               float64
Married              float64
Dependents           float64
Education            float64
Self_Employed        float64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount             int64
Loan_Amount_Term       int64
Credit_History         int64
Property_Area        float64
dtype: object


In [27]:
print(np.isfinite(df.values).all())  # Cela doit renvoyer True
print(np.isnan(df.values).sum())     # Cela doit renvoyer 0


False
3043


In [28]:
# Identifier les valeurs infinies
infinite_mask = ~np.isfinite(df.values)
print("Valeurs infinies par colonne :\n", np.sum(infinite_mask, axis=0))

# Identifier les valeurs NaN
nan_mask = np.isnan(df.values)
print("Valeurs NaN par colonne :\n", np.sum(nan_mask, axis=0))


Valeurs infinies par colonne :
 [500 500 500  43 500 500   0   0   0   0   0 500]
Valeurs NaN par colonne :
 [500 500 500  43 500 500   0   0   0   0   0 500]


In [29]:
# Remplacer les valeurs infinies par NaN pour les traiter ensemble
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Remplacer les valeurs NaN par la moyenne de la colonne
df.fillna(df.mean(), inplace=True)


In [30]:
# Remplacer les valeurs infinies par NaN pour les traiter ensemble
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Supprimer les lignes contenant des valeurs NaN
df.dropna(inplace=True)


In [31]:
# Vérifier les variances des colonnes
variances = df.var()
low_variance_cols = variances[variances == 0].index

# Supprimer les colonnes avec une variance nulle
df.drop(columns=low_variance_cols, inplace=True)


In [32]:
# Créer une instance de la classe CA
afc = CA(row_labels=interested_data.index.values, col_labels=interested_data.columns.values, stats=True)


In [33]:
afc.fit(df.values)

C:\ProgramData\anaconda3\Lib\site-packages\fanalysis\base.py:228: RuntimeWarning: divide by zero encountered in scalar divide
  R = (1.0 / np.sqrt(self.n_)) * (X - E) / np.sqrt(E)
C:\ProgramData\anaconda3\Lib\site-packages\fanalysis\base.py:264: RuntimeWarning: invalid value encountered in divide
  col_coord = ((V.T / np.sqrt(self.c_.T / self.n_))
C:\ProgramData\anaconda3\Lib\site-packages\fanalysis\base.py:295: RuntimeWarning: invalid value encountered in divide
  col_contrib = 100 * ((self.c_.T / self.n_)
C:\ProgramData\anaconda3\Lib\site-packages\fanalysis\base.py:303: RuntimeWarning: invalid value encountered in divide
  / (np.sum((self.n_ / self.c_)
C:\ProgramData\anaconda3\Lib\site-packages\fanalysis\base.py:304: RuntimeWarning: invalid value encountered in divide
  * ((X / self.r_) - (self.c_ / self.n_))**2, axis=1)


CA(col_labels=array(['Married', 'Credit_History'], dtype=object),
   row_labels=array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,...
       416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428,
       429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441,
       442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454,
       455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467,
       468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480,
       481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493,
       494, 495, 496, 497, 498, 499], dtype=int64))

In [41]:
afc.mapping_row(num_x_axis=1,num_y_axis=2)

IndexError: index 0 is out of bounds for axis 1 with size 0

<Figure size 640x480 with 0 Axes>